This code makes a FHIR Patient search request to retrieve a patient record using an NHS number identifier. Here’s a clear, structured description of what the request is doing.

High-level description

The request performs an HTTP GET against a FHIR server’s Patient endpoint, searching for a patient whose identifier matches a given NHS number.
Authentication is handled using HTTP Basic Auth, with credentials loaded from environment variables.

Breakdown of the FHIR request
1. Endpoint being called
GET {FHIR_SERVER}/Patient?identifier=https://fhir.nhs.uk/Id/nhs-number|9999999522


Resource type: Patient

Operation: Search

Search parameter: identifier

2. Search parameter details
identifier=https://fhir.nhs.uk/Id/nhs-number|9999999522


This follows the FHIR search syntax for identifiers:

identifier = system | value


system: https://fhir.nhs.uk/Id/nhs-number
Identifies the namespace for NHS numbers (UK-specific FHIR identifier system)

value: 9999999522
The NHS number being searched for

👉 This tells the FHIR server:

“Return any Patient resource that has an identifier with this system and value.”

In [10]:
import requests
import fhirclient.models.patient as pat
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
load_dotenv()
import os

fhir_password = os.getenv("FHIR_PASSWORD")
fhir_username = os.getenv("FHIR_USERNAME")
server = os.getenv("FHIR_SERVER")

nhsNumber = os.getenv("NHS_NUMBER")
if nhsNumber == None:
    nhsNumber = '9999999522'

api_url = server + "Patient?identifier=https://fhir.nhs.uk/Id/nhs-number|"+nhsNumber
print("Calling GET "+api_url)
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
patientJSON = response.json()
#print(patientJSON)

patientId = None

if ((patientJSON['total']> 0) and (len(patientJSON['entry'])>0)):
    patientId = patientJSON['entry'][0]['resource']['id']
    patient = pat.Patient(patientJSON['entry'][0]['resource'])
    print("Patient Id: " + patientId)
    print("Date of Birth: "+patient.birthDate.isostring)
    for id in patient.identifier:
        print('Identifier: '+ id.value + " " + id.type.coding[0].code)



Calling GET http://192.168.1.67/healthconnect/cdr/fhir/r4/Patient?identifier=https://fhir.nhs.uk/Id/nhs-number|9999999522
Patient Id: 96
Date of Birth: 1945-01-01
Identifier: 9999999522 NH
Identifier: 20002983 MR


In [11]:
import fhirclient.models.diagnosticreport as dr
import fhirclient.models.meta as meta
import pandas as pd
from dateutil import parser

def issued(issued):
    if issued == None:
        return None
    return parser.parse(issued.isostring)

def lastUpdated(meta : meta.Meta):
    if meta == None:
        return None
    return parser.parse(meta.lastUpdated.isostring)

if patientId != None:
    api_url = server + "DiagnosticReport?patient="+patientId+"&_sort=-issued"
    response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
    response1JSON = response.json()

    print(response1JSON['total'])

    diagnosticReports = []
    for entry in response1JSON['entry']:
        #print(entry['resource'])
        print(entry['resource']['resourceType'], entry['resource']['issued'] )
        report = dr.DiagnosticReport(entry['resource'])
        diagnosticReports.append(report)

    dfDR = pd.DataFrame([vars(s) for s in diagnosticReports])
    dfDR['issued'] = dfDR['issued'].apply(issued)
    dfDR['effectiveDateTime'] = dfDR['effectiveDateTime'].apply(issued)
    dfDR['lastUpdated'] = dfDR['meta'].apply(lastUpdated)

dfDR

1
DiagnosticReport 2025-12-06T13:37:35+00:00


,basedOn,category,code,conclusion,conclusionCode,effectiveDateTime,effectivePeriod,encounter,identifier,imagingStudy,...,modifierExtension,text,id,implicitRules,language,meta,_server,_resolved,_owner,lastUpdated
0,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,2025-12-06 13:37:35+00:00,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,99,None,None,<fhirclient.models.meta.Meta object at 0x108a2...,None,None,None,2026-01-07 04:25:42+00:00


In [12]:
import fhirclient.models.documentreference as doc
import pandas as pd
from dateutil import parser


if patientId != None:
    api_url = server + "DocumentReference?patient="+patientId
    response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
    response1JSON = response.json()

    print(response1JSON['total'])

    documents = []
    for entry in response1JSON['entry']:
        document = doc.DocumentReference(entry['resource'])
        documents.append(document)

    dfDOC = pd.DataFrame([vars(s) for s in documents])
    dfDOC['date'] = dfDOC['date'].apply(issued)
    dfDOC['lastUpdated'] = dfDOC['meta'].apply(lastUpdated)
    #dfDOC['effectiveDateTime'] = dfDR['effectiveDateTime'].apply(issued)

dfDOC

1


,authenticator,author,category,content,context,custodian,date,description,docStatus,identifier,...,modifierExtension,text,id,implicitRules,language,meta,_server,_resolved,_owner,lastUpdated
0,None,None,None,[<fhirclient.models.documentreference.Document...,<fhirclient.models.documentreference.DocumentR...,<fhirclient.models.fhirreference.FHIRReference...,2025-12-06 13:37:35+00:00,None,None,[<fhirclient.models.identifier.Identifier obje...,...,None,None,100,None,None,<fhirclient.models.meta.Meta object at 0x10de7...,None,None,None,2026-01-07 04:25:42+00:00


In [13]:
import base64
import io
import tabula
from pypdf import PdfReader

for document in documents:
    content = document.content[0]
    #print(content.attachment.contentType)
    #print(content.attachment.url)
    api_url = server + content.attachment.url
    response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
    response1JSON = response.json()
    encoded = response1JSON['data']
    #print(encoded)
    decode = base64.b64decode(encoded)
    pdf_stream = io.BytesIO(decode)
    pdf = PdfReader(pdf_stream)
    print('Pages: '+ str(len(pdf.pages)))
    print('============ PDF Reader ==============')
    for page in pdf.pages:
        print(page.extract_text())

    tables = tabula.read_pdf(pdf_stream
                             , pages='all', multiple_tables=True)
    print('============ Tabula ==============')
    for table in tables:
        print(table)


Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


Pages: 2
============ PDF Reader ==============
North West Genomic Laboratory Hub (Manchester Site)
Manchester Centre for Genomic Medicine
6th Floor, St Mary's Hospital, Manchester, M13 9WL
Scientific Operational Director: Dr E. Howard
https://mft.nhs.uk/nwglh/  mft.genomics@nhs.net  Tel +44(0) 161 276 6122
 
SevenForename:
EDITTESTPATIENTSurname:
Sex:
01/01/1945 Sample:DoB: Blood
R25-00HN-1Report No:
Hospital No: Not provided
Other No: Not provided
NHS No: 999 999 9522
Collected: Not provided Received: 21/11/2025 15:22 Activated: 21/11/2025
Reported: 06/12/2025
Referred by: Doctor Liverpool, Liverpool Womens Hospital, Liverpool, L8 7SS
Path. No: ACC1973A1
Genomics Laboratory Report
Reason for Testing
RESULT SUMMARY:
Likely pathogenic variant detected.
Likely genetic diagnosis of autosomal dominant XXXX OR GENE-related disease.
This individual is heterozygous for a likely pathogenic GENE variant, VARIANT (details in Appendix II overleaf). Monoallelic pathogenic
GENE variants cause (a r